# Analisi del grafo di Wikipedia Italia 2013
## Progetto finale di Algoritmi e Programmazione per Analisi Dati - A.A. 2023/2024
## Autore: Cristian Bargiacchi

Il progetto consiste nel rispondere a delle domande riguardanti metodi di analisi di grafi viste a lezione, ed è articolato in 3 punti principali. Di seguito importiamo le librerie che serviranno:

In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

Abbiamo due file: il primo, denominato "itwiki-2013.arcs" contiene una riga per ogni arco del nostro grafo. Il secondo, denominato "itwiki-2013.ids" contiene gli ID delle pagine di Wikipedia. La numerazione dei nodi segue gli indici delle righe (alla i-esima riga, numero i). Costruiamo un dizionario che associa i nomi delle pagine al corrispondente ID, ovvero l'indice di riga. Già che ci siamo costruiamo anche il dizionario inverso, che associa ad ogni ID la denominazione della pagina corrispondente (ci servirà in seguito).

In [48]:
path_ids = "itwiki-2013.ids"
page_to_id = {}
id_to_page = {}
with open(path_ids, 'r', encoding='utf-8') as reader_1:
    for i, line in enumerate(reader_1):
        line = line.strip()
        page_to_id[line] = i
        id_to_page[i]=line

print(id_to_page.get(261576))
print(page_to_id.get("Città dell'India"))

Città dell'India
261576


A questo punto procediamo alla costruzione del grafo direzionato su Network X. Per farlo ci conviene prima trasformare il file csv degli archi in un oggetto "Data Frame" della libreria Pandas. Costruiamo anche una versione ridotta di 10.000 archi per usarlo come prova.

In [40]:
path_arcs = "itwiki-2013.arcs"
arcs_df_prova = pd.read_csv(path_arcs,sep=" ", header=None, names=['v1','v2'])
# arcs_df_prova = arcs_df[:50000] # per la prova

A questo punto procediamo alla costruzione del grafo direzionato.

In [41]:
def create_graph(df:pd.DataFrame) -> nx.DiGraph:
    G = nx.DiGraph()
    
    for index, line in df.iterrows():
        v1 = int(line['v1'])
        v2 = int(line['v2'])
        G.add_edge(v1,v2)
        
    return G
g_prova = create_graph(arcs_df_prova)
print(g_prova)

DiGraph with 1016179 nodes and 25619926 edges


# Domanda 1: Calcola la distribuzione "out-degree" del grafo

Per rispondere alla prima domanda definiremo una funzione "outDegree_distribution". Essa costruirà un dizionario che associa ad ogni nodo la lista dei suoi vicini uscenti: il grado in uscita del nodo sarà la lunghezza di questa lista.
A questo punto sarebbe sufficiente associare a ogni grado le frequenze di nodi aventi quel valore. Tuttavia oltre alla frequenza associamo al grado anche la lista dei nodi con quel valore di out degree, per poterli utilizzare in seguito. 

In [42]:
def outDegree_distribution (g:nx.DiGraph) -> dict:
    neighbors_dict ={node: list(g.successors(node)) for node in g.nodes()} # da nodo a lista vicini
    distribution_dict = {} # da grado a array di 2 elementi: in posizione 0, nodi con quel grado; in posizione 1, conteggio nodi con quel grado
    for node in neighbors_dict.keys():
        degree_node = len(neighbors_dict[node])
        if degree_node in distribution_dict:
            distribution_dict[degree_node][0].append(node)
            distribution_dict[degree_node][1]+=1
        else:
            distribution_dict[degree_node] = [[node],1]
    return distribution_dict
g_outDegree = outDegree_distribution(g_prova)

Plottiamo la distribuzione delle frequenze degli out-degree:

In [ ]:
degrees = list(g_outDegree.keys())
frequencies = [g_outDegree[degree][1] for degree in degrees]

plt.bar(degrees, frequencies)
plt.xlabel('Out-degree')
plt.ylabel('Frequency')
plt.xlim([0,200])
plt.ylim([0,max(frequencies)])
plt.title('Out-degree Distribution')
plt.show()

## 1.1: Quali sono le 10 pagine di Wikipedia con maggior numero di link verso altre pagine?

Per trovare i 10 nodi con grado in uscita maggiore, aggiungiamo tutti i nodi con grado massimo a una lista finchè questa non arriva a lunghezza 10. Partiamo dal grado massimo e ad ogni aggiunta decrementiamo di 1. Abbiamo scelto questa strada in quanto l'ordinamento dei nodi per grado decrescente sarebbe stato leggermente più costoso a livello computazionale rispetto a calcolarne il massimo 10 volte. 

In [44]:
max_degree = max(degrees)
top_10 = list()

while len(top_10)<10:
    if max_degree in g_outDegree.keys():
        nodes = list(g_outDegree[max_degree][0])
        top_10.extend(nodes)
    max_degree = max_degree - 1
top10 = top_10[:10]
top10_pages = []
for id in top10:
    top10_pages.append(id_to_page[id])
top10_pages = pd.Series(top10_pages, index=range(1,11))
print(top10_pages)

1                  Città dell'India
2     Classificazione Nickel-Strunz
3                     Nati nel 1981
4                     Nati nel 1985
5                     Nati nel 1983
6                     Nati nel 1984
7                     Nati nel 1986
8                     Nati nel 1982
9                     Nati nel 1980
10                Nativi del Veneto
dtype: object


# Domanda 2: Sia U(G) la versione indiretta del grafo. Calcola il diametro della sua componente connessa più grande.

Per il calcolo del diametro utilizzeremo l'algoritmo "iFub" visto a lezione. Prima di tutto definisco una funzione per rendere il grafo indiretto e selezionare la componente connessa più grande.

In [50]:
def get_largest_cc_undirected(G:nx.DiGraph)->nx.Graph:
    G = nx.to_undirected(G)
    return G.subgraph(
    sorted(nx.connected_components(G), key = len, reverse=True)[0]
    ).copy()

u_g_largest_cc = get_largest_cc_undirected(g_prova)

print(u_g_largest_cc)

Graph with 1016177 nodes and 23436213 edges


In [51]:
def calculate_F(G:nx.Graph,node:int,distance:int)->set:
    return nx.descendants_at_distance(G,node,distance) # conviene fare la bfs a mano e poi fare un dizionario a cui associo il la distanza

def calculate_B_i(G:nx.Graph, node:int, i:int)->int:
    F = calculate_F(G,node,distance=i)
    B_i = 0
    for node in F:
        max = nx.eccentricity(G, v=node)
        if max > B_i:
            B_i = max
    return B_i

In [52]:
def iFub(G:nx.DiGraph)-> int:
    G = get_largest_cc_undirected(G)
    starting_node = page_to_id[top10_pages[1]]
    i = nx.eccentricity(G,v=starting_node)
    lb = i
    ub = 2*lb
    
    while ub > lb:
        B_i = calculate_B_i(G,starting_node,i)
        max = np.max([lb,B_i])
        if max > 2*(i-1):
            return max
        else:
            lb = max
            ub = 2*(i-1)
        i=i-1
    return lb

diameter = iFub(g_prova)
print("diameter iFub: ", diameter)

diameter iFub:  8


In [54]:
diameter_nx = nx.diameter(u_g_largest_cc)
print("diameter nx: ", diameter_nx)

KeyboardInterrupt: 

In [ ]:
def create_graph_without_disambigua(df:pd.DataFrame) -> nx.DiGraph:
    G = nx.DiGraph()
    
    for index, line in df.iterrows():
        v1 = int(line['v1'])
        v2 = int(line['v2'])
        
        if "disambigua" not in id_to_page[v1] and "disambigua" not in id_to_page[v2]:
            G.add_edge(v1,v2)
    return G

In [ ]:
G_without_disambigua_prova = create_graph_without_disambigua(arcs_df_prova)
print(G_without_disambigua_prova)

In [ ]:
diameter_without_disambigua = iFub(G_without_disambigua_prova)
print("diameter iFub: ", diameter_without_disambigua)

In [ ]:
u_g_without_disambigua = get_largest_cc_undirected(G_without_disambigua_prova)
print(nx.diameter("diameter nx: ", u_g_without_disambigua))

Trovare una clique massimale nel grafo indiretto U(g). 
Clique: sottografo completo (ogni nodo è connesso ad ogni altro). Clique massimale: aggiungendo un altro nodo smetterebbe di essere una clique. Idea: parto da un clique vuota, aggiungo un nodo e verifico se è ancora una clique. L'aggiunta dei nodi avviene col metodo della visita in profondità (DFS), poichè valuto un nodo alla volta vicino di uno precedentemente valutato.



In [ ]:
def Bron_Kerbosch(G, R, P, X):
    if not P and not X:
        return R
    for v in list(P):
        myclique = Bron_Kerbosch(
            G, 
            R.union({v}), 
            P.intersection(G.neighbors(v)), 
            X.intersection(G.neighbors(v))
        )
        if myclique and len(myclique)>3:
            return myclique
        P.remove(v)
        X.add(v)
    return None

In [ ]:
from random import choice 
def find_maximal_clique(G):
    G = nx.to_undirected(G)
    nodes = list(G.nodes())
    
    for i in range(len(nodes)):
        start_node = choice(nodes)
        R = {start_node}
        P = set(G.neighbors(start_node))
        X = set()
        myclique = Bron_Kerbosch(G, R, P, X)
        if myclique and len(myclique) > 3:
            myclique_pages = set()
            for id in myclique:
                myclique_pages.add(id_to_page[id])
            return myclique_pages
    
    return None  # se nessuna clique di ordine almeno 3 è trovata

maximal_clique = find_maximal_clique(g_prova)
print("Una clique massimale è composta da:", maximal_clique)

